In [1]:
import numpy as np
from scipy.sparse import rand as sprand
from scipy.sparse import lil_matrix
import torch
from torch.autograd import Variable
import pandas as pd

In [45]:
resultsFile = pd.read_csv('LambdaMeanLossResults.csv',index_col=0)

In [46]:
resultsFile

,0.001,0.01,0.1
r1,1.72851,1.72088,1.67337
r2,1.71202,1.70354,1.65576
r3,1.71550,1.70717,NaN
r4,NaN,NaN,NaN
r5,NaN,NaN,NaN


In [2]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
print("Loading Data")
df_train = pd.read_csv('ml-10M100K/r3.train', sep='::', names=names,engine='python')
print("Loaded Train")
df_test = pd.read_csv('ml-10M100K/r3.test', sep='::', names=names,engine='python')
print("Loaded Test")

Loading Data
Loaded Train
Loaded Test


In [3]:
df_train.head()

,user_id,item_id,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


In [4]:
df_train.head()
print(df_train.shape)

(8000044, 4)


In [5]:
def get_movielens_ratings(df):
    n_users = max(df.user_id.unique())
    n_items = max(df.item_id.unique())

    interactions = lil_matrix( (n_users,n_items), dtype=float) #np.zeros((n_users, n_items))
    for row in df.itertuples():
        interactions[row[1] - 1, row[2] - 1] = row[3]
    return interactions

In [6]:
print("Preparing Training Data")
ratings = get_movielens_ratings(df_train)
print(ratings.shape)

Preparing Training Data
(71567, 65133)


In [7]:
print("Preparing Testing Data")
test_ratings = get_movielens_ratings(df_test)
print(test_ratings.shape)

Preparing Testing Data
(42788, 65133)


In [20]:
class MatrixFactorization(torch.nn.Module):


    def __init__(self, n_users, n_items, n_factors=5, useBias = False):
        super().__init__()
        
        if torch.cuda.is_available(): # CHECK FOR CUDA AVAILABILITY
            self.useCUDA = True
            print("CUDA is being used")
        else:
            print("CUDA not available, reverting to CPU")
            
        if self.useCUDA: # IF IT IS AVAILABLE, USE IT
            self.cuda() 
        
        self.user_factors = torch.nn.Embedding(n_users, 
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items, 
                                               n_factors,
                                               sparse=False)
        # Also should consider fitting overall bias (self.mu term) and both user and item bias vectors
        
        ## Incorporation of Bias Term
        if useBias:
            self.user_bias = torch.nn.Embedding(n_users, 1, sparse = False)
            self.item_bias = torch.nn.Embedding(n_itms, 1, sparse = False)
        
        
        # Mu is 1x1, user_bias is 1xn_users. item_bias is 1xn_items
        
    def getLoss(self):
        return self.currentLoss.data.cpu().numpy()[0]


    currentLoss = 10
    useCUDA = False
    
    interactions = False
#     loss_func = torch.nn.MSELoss()
#     reg_loss_func = torch.optim.SGD(self.model.parameters(), lr=1e-6, weight_decay=1e-3)
    
    # For convenience when we want to predict a single user-item pair. 
    def predict(self, user, item):
        # Need to fit bias factorsx
        print("Predicting")
        
        print(self.user_factors(user))
        print(torch.transpose(self.item_factors(item),0,1))

#         return torch.mm(self.user_factors(user)[0],torch.transpose(self.item_factors(item),0,1)[0])
        return torch.mm(self.user_factors(user),torch.transpose(self.item_factors(item),0,1))
    
    
#         return torch.dot(self.user_factors(user),self.item_factors(item))
    
    # Much more efficient batch operator. This should be used for training purposes
    def forward(self, users, items):
        # Need to fit bias factors
#         print("Forward")
        return torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
    
    def get_batch(self,batch_size,ratings):
        # Sort our data and scramble it
        rows, cols = ratings.shape
        p = np.random.permutation(rows)

        # create batches
        sindex = 0
        eindex = batch_size
        while eindex < rows:
            batch = p[sindex:eindex]
            temp = eindex
            eindex = eindex + batch_size
            sindex = temp
            yield batch

        if eindex >= rows:
            batch = range(sindex,rows)
            yield batch 
    
    def run_test(self,batch_size,ratings_test):

        predictionsArray = []
        losses = []
        print("Number Batches: {}".format(ratings_test.shape[0]/batch_size))
        for i,batch in tqdm(enumerate(model.get_batch(batch_size,ratings_test))):
            print(i)
            if self.useCUDA:
                interactions = Variable(torch.cuda.FloatTensor(ratings_test[batch, :].toarray()),volatile = True)
                rows = Variable(torch.cuda.LongTensor(batch), volatile = True)
                cols = Variable(torch.cuda.LongTensor(np.arange(ratings_test.shape[1])), volatile = True)               
            else:
                interactions = Variable(torch.FloatTensor(ratings_test[batch, :].toarray()))
                rows = Variable(torch.LongTensor(batch))
                cols = Variable(torch.LongTensor(np.arange(ratings_test.shape[1])))

            # Predict and calculate loss
            predictions = self(rows, cols)
            # predictions = self.predict(rows,cols)
#              print(type(predictions))
            predictionsArray.append(predictions.data.cpu().numpy())
            # print(type(self.loss_func(predictions, interactions)))
            losses.append(self.loss_func(predictions, interactions).data.cpu().numpy())

            # del interactions
            # del rows
            # del cols
        return predictionsArray, losses
    
    def run_epoch(self,batch_size, ratings):
        for i,batch in tqdm(enumerate(self.get_batch(batch_size, ratings))):
            # Set gradients to zero
            self.reg_loss_func.zero_grad()

#             print(type(batch))
            # Turn data into variables
            if self.useCUDA:
#                 print("using cuda")
                interactions = Variable(torch.cuda.FloatTensor(ratings[batch, :].toarray()))
                rows = Variable(torch.cuda.LongTensor(batch))
                cols = Variable(torch.cuda.LongTensor(np.arange(ratings.shape[1])))               
            else:
                interactions = Variable(torch.FloatTensor(ratings[batch, :].toarray()))
                rows = Variable(torch.LongTensor(batch))
                cols = Variable(torch.LongTensor(np.arange(ratings.shape[1])))

#             print(type(rows))
#             print(type(cols))
            # Predict and calculate loss
            predictions = model(rows, cols)
#             print(predictions)
            self.currentLoss = self.loss_func(predictions, interactions)

            # Backpropagate
            self.currentLoss.backward()

            # Update the parameters
            self.reg_loss_func.step()
    
    def train(self, numEpochs, batch_size, ratings,learningRate):
        self.loss_func = torch.nn.MSELoss()
        self.reg_loss_func = torch.optim.SGD(model.parameters(), lr=learningRate, weight_decay=1e-3)
        for i in range(numEpochs):
            print(i)
            self.run_epoch(batch_size,ratings)
            
    def convertArryToVariable(self, array):
        if self.useCUDA:
            return Variable(torch.cuda.LongTensor(array))
        else:
            return Variable(torch.LongTensor(array))


    
    def convertLillMatrixToVariable(self,lillMatrix):
        if self.useCUDA:
            if lillMatrix.shape[0] == 1:
                # we have a single matrix
                return Variable(torch.cuda.LongTensor(lillMatrix.toarray()))
            else:
                return Variable(torch.cuda.LongTensor(lillMatrix.toarray()))
        else:
            if lillMatrix.shape[0] == 1:
                # we have a single matrix
                return Variable(torch.LongTensor(lillMatrix.toarray()))
            else:
                return Variable(torch.LongTensor(lillMatrix.toarray()))


In [21]:
print("Creating Model")
model = MatrixFactorization(ratings.shape[0], ratings.shape[1], n_factors=2,useBias = False)
if torch.cuda.is_available():
    model.cuda()

Creating Model
CUDA is being used


In [22]:
EPOCH = 1
BATCH_SIZE = 1000 #50
LR = 0.001

In [23]:
print("Training Model")
model.train(EPOCH,BATCH_SIZE,ratings,LR)

Training Model
0


In [24]:
print("Model Loss: {}".format(model.getLoss()))

Model Loss: Variable containing:
 1.9848
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [ ]:
#Task 1:

loadPrevResults = False
loadSavedMeans = False

EPOCH = 1 # Number of Epochs to train for
BATCH_SIZE = 1000 #50
LRs = [0.001, 0.01, 0.1] # array of learning rates to test

FileNames = ["r4"]
if loadPrevResults:
    LambdaMeanLossResultsFrame = pd.read_csv('Task1_LambdaMeanLossResults_{}Epochs.csv'.format(EPOCH),index_col=0)

    print(LambdaMeanLossResultsFrame)
else:
    LambdaMeanLossResultsFrame = pd.DataFrame(index=["r1","r2","r3","r4","r5"], columns=['0.001','0.01','0.1'])
    print(LambdaMeanLossResultsFrame)



print("Running Exmperiment on file ids {}".format(FileNames))
for fileName in FileNames:

    print("Starting Building model for cross validation {}".format(fileName))
    names = ['user_id', 'item_id', 'rating', 'timestamp']
    print("Loading Data")
    df_train = pd.read_csv('ml-10M100K/'+fileName+'.train', sep='::', names=names,engine='python')
    print("Loaded train data")
    df_test = pd.read_csv('ml-10M100K/'+fileName+'.test', sep='::', names=names,engine='python')
    print("Loaded Data")

    ratings = get_movielens_ratings(df_train)
    print(ratings.shape)

    test_ratings = get_movielens_ratings(df_test)
    print(test_ratings.shape)

    print("Prepared the data")


    print("Creating Model")
    model = MatrixFactorization(ratings.shape[0], ratings.shape[1], n_factors=2,useBias = False)
    if torch.cuda.is_available():
        model.cuda()

    print("Running the training")

    print("Learining Rates tested: {}".format(LRs))
    for LR in LRs:
        print("Training Model")
        print("Number iterations Per Epoch: {}".format(ratings.shape[0]/BATCH_SIZE))
        model.train(EPOCH,BATCH_SIZE,ratings,LR)

        print("Model Loss: {}".format(model.getLoss()))
        print("Running Test")
        predictions, losses = model.run_test(10,test_ratings)

        print(len(predictions))
        print(np.mean(losses))

        LambdaMeanLossResultsFrame.loc[fileName][str(LR)] = np.mean(losses)
        print(LambdaMeanLossResultsFrame)

        LambdaMeanLossResultsFrame.to_csv('Task1_LambdaMeanLossResults_{}Epochs.csv'.format(EPOCH))

    LambdaMeanLossResultsFrame.to_csv('Task1_LambdaMeanLossResults_{}Epochs.csv'.format(EPOCH))

LambdaMeanLossResultsFrame.to_csv('Task1_LambdaMeanLossResults_{}Epochs.csv'.format(EPOCH))

print(LambdaMeanLossResultsFrame)